As Formulaic matures it is expected that it will be integrated directly into
downstream projects where formula parsing is required. This is known to have
already happened in the following high-profile projects:

* [lifelines](https://lifelines.readthedocs.io/)
* [linearmodels](https://bashtage.github.io/linearmodels/)
* Others?

Where direct integration has not yet happened, you can still use Formulaic in
conjunction with other commonly used libraries. On this page, we will add
various examples how to achieve this. If you have done some integration work,
please feel free to submit a PR that extends this documentation!

## StatsModels

[statsmodels](https://www.statsmodels.org/stable/index.html) is a popular 
toolkit hosting many different statistical models, tests, and exploration tools.
The formula API in `statsmodels` is currently based on `patsy`. If you need the
features found in Formulaic, you can use it directly to generate the model 
matrices, and use the regular API. For example:

In [1]:
import pandas
from formulaic import model_matrix
from statsmodels.api import OLS
data = pandas.DataFrame({"y": [0.1, 0.4, 3], "a": [1,2,3], "b": ["A", "B", "C"]})
y, X = model_matrix("y ~ a + b", data)
model = OLS(y, X)
results = model.fit()
print(results.summary())



                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Fri, 14 Apr 2023   Prob (F-statistic):                nan
Time:                        21:15:59   Log-Likelihood:                 102.01
No. Observations:                   3   AIC:                            -198.0
Df Residuals:                       0   BIC:                            -200.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.7857        inf         -0        n

/home/matthew/.pyenv/versions/3.11.2/lib/python3.11/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
/home/matthew/.pyenv/versions/3.11.2/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1765: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/matthew/.pyenv/versions/3.11.2/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1765: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/home/matthew/.pyenv/versions/3.11.2/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1687: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


## Scikit-Learn

[scikit-learn](https://scikit-learn.org/) is a very popular machine learning
toolkit for Python. You can use Formulaic directly, as for `statsmodels`, or 
as a module in scikit-learn pipelines along the lines of:

In [2]:
from typing import Optional

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from formulaic import Formula, FormulaSpec, ModelSpec


class FormulaicTransformer(TransformerMixin, BaseEstimator):

    def __init__(self, formula: FormulaSpec):
        self.formula: Formula = Formula.from_spec(formula)
        self.model_spec: Optional[ModelSpec] = None
        if self.formula._has_structure:
            raise ValueError(f"Formula specification {repr(formula)} results in a structured formula, which is not supported.")

    def fit(self, X, y = None):
        """
        Generate the initial model spec by which subsequent X's will be 
        transformed.
        """
        self.model_spec = self.formula.get_model_matrix(X).model_spec
        return self

    def transform(self, X, y = None):
        """
        Transform `X` by generating a model matrix from it based on the fit
        model spec.
        """
        if self.model_spec is None:
            raise RuntimeError("`FormulaicTransformer.fit()` must be called before `.transform()`.")
        X_ = self.model_spec.get_model_matrix(X)
        return X_


pipe = Pipeline([
    ("formula", FormulaicTransformer("x1 + x2 + x3")),
    ("model", LinearRegression())
])
pipe_fit = pipe.fit(pandas.DataFrame({"x1": [1,2,3], "x2": [2, 3.4, 6], "x3": [7, 3, 1]}), y=pandas.Series([1,3,5]))
pipe_fit
# Note: You could optionally serialize `pipe_fit` here.
# Then: Use the pipe to predict outcomes for new data.


Pipeline(steps=[('formula', FormulaicTransformer(formula=1 + x1 + x2 + x3)),
                ('model', LinearRegression())])